# Introduction 

This tutorial will introduce you to Johnson's algorithm. This algorithm uses Dijkstra's algorithm and Bellman-Ford's algorithm to solve the all pairs shortest path problem (APSP). The APSP problem is important in data science because it allows to for finding all shortest paths between nodes in a network and is also used to solve Network Flow problems. 


## Tutorial content

In this tutorial, I will explain how the algorithm works, show how to implement Johnson's algorithm in Python and then run an example using the algorithm on a weighted graph.

The library we will need to use for this is heapq, which allows us to use a min heap for a priority queue. To import this library run the following cell:

In [27]:
import heapq

## How does Johnson's Algorithm work

Johnson's algorithm works on all graphs as long as the edges do not have negative cycles in the graph. It allows for negative weights and is significantly more efficient than simply running Bellman-Ford’s algorithm from each source. It uses a clever trick to tweak the weights and eliminate negative ones without altering the shortest paths. Johnson's algorithm first uses Bellman-Ford to get values to adjust the weights of the paths and the after runs Dijkstra's algorithm on all vertices to get shortest path from every vertex.

### Dijkstra's Algorithm

Dijkstra's Algorithm is a single source shortest path algorithm that we have discussed in class where you start from a single vertex and get the shortest path to all other vertices (note this algorithm only works on positive edge weights). Previously when discussed in class, we talked about all edge weights having a weight of 1 but we will extend this and talk about how Dijkstra's works with edge weights greater than or equal to 1 and have directed edges. 

As noted above, all edges must be positive, this means that there cannot be a negative-weight cycle since there are no negative weights in the first place. Therefore, you can never make a path shorter by visiting a vertex twice. This is because cycling back to a vertex cannot have less weight than the path that ends at the first vist to the vertex. As a result, when searching for the shortest path we only need to consider simple paths with no repeated vertiecs. 

In Dijkstra's algorithm we exploit this property by keeping track of which vertices we have visited and we use a priority queue that keeps track of the potential shortest distences. When we take a vertex off the priority queue, we add all the neighbors to the priority queue with the distance from the parent plus the edges weight connecting it as the priority weight.

Essentially, what this is doing is that we are expanding out from the source, always expanding the remaining node that is closest to the source, and updating the distance for all the neighbors of this node.

As discussed in the lecture notes, the complexity of Dijkstra’s algorithm is dependent on the complexity of the inserting and deleting from the priority queue. At a high leve with the number of vertices being n and the number of edges being m, this will make about O(n+m) queries since we iterate over each vertex and iterate over all the edges the actual complexity with a well-designed priority queue is O(nlog(n)+m) as mentioned in the lecture notes. This can be done by using a priority queues that supports a decreaseKey operation. This leads to 𝑂(m+nlog(n)) work across all priority queue operations, which will not be gone into detail here. But with a priority queue with O(log n) inserts and deletes, this will be O(mlog(n)) work.

### Code for Dijkstra's Algorithm

For this algorithm let V be a set or list of all the vertices, in our case lets make them 1 to n. Let E be the set of tuples being (start vertex, end vertex, weight of edge) so for instance if the graph has an edge from 1 to 2 with weight 10, E would contain the tuple (1,2,10).

In [90]:
def Dijkstra(V, E, source):
    distances = {}
    graph = {}
    seen = set()
    n = len(V)
    
    for v in V:
        distances[v] = float("inf")
        graph[v] = {}
        
    distances[source] = 0
        
    for (u,v,w) in E: # creates a dictionary that is in the format of an adjaceny list
        graph[u][v] = w
            
    pq = []
    heapq.heappush(pq, (0, source))
    while (len(seen) != n and len(pq) != 0):
        dist, vertex = heapq.heappop(pq)
        if vertex in seen:
            continue # This is because as discussed above, if you have to visit a vertex twice then the path is not any shorter
        seen.add(vertex)
        
        for neighbor in graph[vertex]:
            neighbor_distance = dist + graph[vertex][neighbor]
            if neighbor_distance < distances[neighbor]:
                distances[neighbor] = neighbor_distance
                heapq.heappush(pq, (neighbor_distance, neighbor))
                
    return distances

Let's run Dijkstra's on the graph below by hand to see how it works and then run the code to make sure it is correct.

![image](graph2.png)

We start off at vertex 1. Since this is the source node, we add it to the dictionary with a distance of 0. We then add all of its neighbors to the priority queue with a distance of 0 plus the weight of the edge from the source to the neighbors and set it to seen. As it goes through the neighbors, it will see if the new distances are shorter than the current distance from the source and if it is it will update the distance. So without loss of generality (WLOG)  suppose it looks at neighbor 2 first. Since it has not beed seen yet, it has a distance of infinity. Since 1 < infinity, it updates the distances dictionary to set the distance of 2 to be 1 and adds (1, 2) to the priority queue. Similarly it does this with vertex 4 and updates the distance to 3 and adds (3, 4) to the priority queue. Then it updates 5 in the dictionary to have a distance of 100 and adds (100, 5) to the priority queue. At this point, in order, the priority queue has (1,2), (3, 4), (100, 5). The distances dictionary is {1:0, 2:1, 3:infinity, 4:3, 5:100}.

Since (1,2) is at the top we pop it off the queue. Since it has not been marked as seen, we add it to the set of seen vertices and go through its neighbors. Its only neighbor is 3. Since the distance is 2, which is 1 plus the edge weight from 2 to 3 which is 5 is less than infinity, we update the distance for 3 to be 1+5=6 and add (6, 3) to the priority queue. So now the priority queue is (3, 4), (6, 3), (100, 5). The distances dictionary is {1:0, 2:1, 3: 6, 4: 3, 5:100}.

Next we pop off (3, 4) from the priority queue. Since we have not seen 4 before we add it to the set of seen vertices and look at its neighbors. The set of neighbors are 3 and 5. Lets start with 3. Since the distance to 3 in the distances dictionary is 6 as specified above but the distance to 4 which is 3 + the distance from 4 to 3 is 2 this is a total of 5 which is less than the distance in the dictionary. Thus we update the dictionary to have a distance of 5 and add (5, 3) to the priority queue. Then we look at 5 and see that the distance in the dictionary is 100 which is greater than the distance to 4 which is 3 + the distance from 4 to 5 is 6 so a total distance of 9 so we update the dictionary to have the distance to 5 be 9 and add it to the priority queue. So now the priority queue is (5, 3), (6, 3), (9, 5), (100, 5). The distances dictionary is {1:0, 2:1, 3: 5, 4: 3, 5:9}.

Next we look at (5, 3). Since we have not seen 3 before we add it to the set of seen vertices and look at its neighbors. The only neighbor is 5. We look at 5 and see that the distance in the dictionary is 9 which is greater than the distance to 3 which is 5 + the distance from 3 to 5 is 1 = 6 so we update the dictionary to have the distance to 5 be 6 and add it to the priority queue. So now the priority queue is (6, 3), (6, 5), (9, 5), (100, 5). The distances dictionary is {1:0, 2:1, 3: 5, 4: 3, 5:6}.

We pop off (6, 3) from the priority queue next, but since 3 is in the set of seen vertices we skip it. So now the priority queue is (6, 5), (9, 5), (100, 5). The distances dictionary is {1: 0, 2: 1, 3: 5, 4: 3, 5: 6}.

Next we look at (6, 5) since the len seen is less than 5. We add it to the set seen but it has no neighbors so we finish this iteration. So now the priority queue is (9, 5), (100, 5). The distances dictionary is {1: 0, 2: 1, 3: 5, 4: 3, 5: 6}.

But now the set seen has 5 vertices and thus we are done running Dijkstra's algorithm on this graph.

In [110]:
result = Dijkstra({1,2,3,4,5}, {(1,5,100), (1,4,3), (1,2,1), (2,3,5), (3,5,1), (4,3,2), (4,5,6)}, 1)
print("Graph 2 results")
for key in result:
    print(f"from vertex 1 to {key} is {result[key]}")

Graph 2 results
from vertex 1 to 1 is 0
from vertex 1 to 2 is 1
from vertex 1 to 3 is 5
from vertex 1 to 4 is 3
from vertex 1 to 5 is 6


Here is another example. We will not run through it but you should do so on your own to make sure your understand the algorithm.

![image](graph.png)

In [111]:
result = Dijkstra({1,2,3,4,5}, {(1,5,2), (1,2,5), (2,5,1), (1,3,4), (2,3,4), (3,2,4), (3,4,2), (4,3,2), (1,5,2), (3,5,2), (4,5,2)}, 1)
print("Graph 1 results")
for key in result:
    print(f"from vertex 1 to {key} is {result[key]}")
print()

Graph 1 results
from vertex 1 to 1 is 0
from vertex 1 to 2 is 5
from vertex 1 to 3 is 4
from vertex 1 to 4 is 6
from vertex 1 to 5 is 2



### Bellman-Ford Algorithm

Bellman-Ford is a single source shortest path problem but can be ran on graphs with negative edge weights. In addition it will detect if there are negative-cycles in the graph, which as noted above is something that Johnson's algorithm does not deal with. Dijkstra's algorithm works by exploiting the property that once we reach a vertex from a shortest path, we do not need to go through any vertex twice and we can build longer paths from shorter paths. We can do something similar with Bellman-Ford's algorithm. Rather than adding more and more vertices we can add more and more edges. 

Suppose we have found a path to a vertex throught k or fewer edges. We can find out the shortest path using k+1 edges by taking all the paths using the k edges or fewer and add one more edge if it makes the path shorter for another vertex or leave it unchanged otherwise since we do not want to lengthen the paths. If the graph does not have any negative-weight cycles, then all shortest paths have |V|−1 or fewer edges. Thus we only have to repeat this process at most |V| times. If we repeat the process for a |V|+1 time and the path distances changes this means there is a negative cycle.

The complexity of the Bellman-Ford algorithm where n = |V| and m = |E| is O(nm) since the number of iterations will be n and goes through all the edges. Depending on the implementation, the runtime could be multiplied by log(n) since that is the cost of the priority queue operations which would result in a O(nmlog(n)) runtime but in our algorithm we will not be using any priority queue, therefore the run time will be O(nm).

### Code for the Bellman-Ford Algorithm

In [93]:
def BellmanFord(V, E, source):
    distances = {}
    n = len(V)
    
    for v in V:
        distances[v] = float("inf")
    distances[source] = 0
        
    oneChanged = False
    for i in range(n):
        for (u, v, w) in E:
            temp = min(distances[u] + w, distances[v])
            if (temp != distances[v]):
                oneChanged = True
            distances[v] = min(distances[u] + w, distances[v])
        if oneChanged == False:
            return distances
    
    for (u, v, w) in E:
            if distances[u] + w < distances[v]:
                return None #to signify there is a negative cycle
             
    return distances

![image](graph3.png)

Lets run through this graph by hand.

If we have the source vertex of 1 and the edges of {(1, 2, -1), (1, 3, 4), (2, 4, 2), (2, 5, 2), (5, 4, -3), (4, 3, 5), (2, 3, 3)} and starting distances of {1: 0, 2: infinity, 3: infinity, 4: infinity, 5: infinity}.

For the first iteration we will loop through all of the vertices. Since all except the source start off with a distance of 0, doing distance\[u\] + w for any weight will be infinity and thus not less than distance\[v\] unless u = 1 then all of the distances will update. So in this case we have edges (1, 2, -1) and (1, 3, 4) that will update the weights. Since we have distance\[2\] = min(distance\[1\]-1, distance\[2\]) = min(0-1, infinity) this will update distance\[2\] to be -1. Similarly with (1, 3, 4), since distance\[3\] = min(distance\[1\]+4, distance\[3\]) = min(0+4, infinity) = 4 this will update distance\[3\] = 4. So now the distances will be {1: 0, 2: -1, 3: 4, 4: infinity, 5: infinity}.

For the second iteration we once again loop through all the edges.

(1, 2, -1) -> new_distances\[2\] = min(distance\[1\]-1, distance\[2\]) = min(0-1, -1) = -1

(1, 3, 4) -> new_distances\[3\] = min(distance\[1\]+4, distance\[3\]) = min(0+4, 4) = 4

(4, 3, 5) -> new_distances\[3\] = min(distance\[4\]+5, distance\[3\]) = min(infinity+5, 4) = 4

(2, 3, 3) -> new_distances\[3\] = min(distance\[2\]+3, distance\[3\]) = min(-1+3, 4) = 2

(2, 4, 2) -> new_distances\[4\] = min(distance\[2\]+2, distance\[4\]) = min(-1+2, infinity) = 1 

(5, 4, -3) -> new_distances\[4\] = min(distance\[5\]-3, distance\[4\]) = min(infinity-3, 1) = 1

(2, 5, 2) -> new_distances\[5\] = min(distance\[2\]+2, distance\[5\]) = min(-1+2, infinity) = 1

So the new dictionary of distances is {1: 0, 2: -1, 3: 2, 4: 1, 5: 1}.


For the third iteration:

(1, 2, -1) -> new_distances\[2\] = min(distance\[1\]-1, distance\[2\]) = min(0-1, -1) = -1

(1, 3, 4) -> new_distances\[3\] = min(distance\[1\]+4, distance\[3\]) = min(0+4, 2) = 2

(4, 3, 5) -> new_distances\[3\] = min(distance\[4\]+5, distance\[3\]) = min(1+5, 2) = 2

(2, 3, 3) -> new_distances\[3\] = min(distance\[2\]+3, distance\[3\]) = min(-1+3, 2) = 2

(2, 4, 2) -> new_distances\[4\] = min(distance\[2\]+2, distance\[4\]) = min(-1+2, 1) = 1 

(5, 4, -3) -> new_distances\[4\] = min(distance\[5\]-3, distance\[4\]) = min(1-3, 1) = -2

(2, 5, 2) -> new_distances\[5\] = min(distance\[2\]+2, distance\[5\]) = min(-1+2, -2) = 1

So the new dictionary of distances is {1: 0, 2: -1, 3: 2, 4: -2, 5: 1}.


For the fourth iteration it is the same as the 3rd:

(1, 2, -1) -> new_distances\[2\] = min(distance\[1\]-1, distance\[2\]) = min(0-1, -1) = -1

(1, 3, 4) -> new_distances\[3\] = min(distance\[1\]+4, distance\[3\]) = min(0+4, 2) = 2

(4, 3, 5) -> new_distances\[3\] = min(distance\[4\]+5, distance\[3\]) = min(-2+5, 2) = 2

(2, 3, 3) -> new_distances\[3\] = min(distance\[2\]+3, distance\[3\]) = min(-1+3, 2) = 2

(2, 4, 2) -> new_distances\[4\] = min(distance\[2\]+2, distance\[4\]) = min(-2+2, -2) = -2 

(5, 4, -3) -> new_distances\[4\] = min(distance\[5\]-3, distance\[4\]) = min(1-3, -2) = -2

(2, 5, 2) -> new_distances\[5\] = min(distance\[2\]+2, distance\[5\]) = min(-1+2, -2) = 1

So the new dictionary of distances is {1: 0, 2: -1, 3: 2, 4: -2, 5: 1}.

Since the distances in the fourth iteration are the same as teh distances in the third iteration this signifies that they will not change and we can return the distances. This is essentially "short circuiting" the algorithm because this signifies the distances will not get any shorter.

Since the last two iterations were the same when it will return the dictionary with {1: 0, 2: -1, 3: 2, 4: -2, 5: 1}. When we run the Bellman-Ford algorithm with this set of edges and source of 1, we will get the same values.

In [94]:
result = BellmanFord({1,2,3,4,5}, {(1, 2, -1), (1, 3, 4), (2, 4, 2), (2, 5, 2), (5, 4, -3), (4, 3, 5), (2, 3, 3)}, 1)
print(result)

{1: 0, 2: -1, 3: 2, 4: -2, 5: 1}


For practice, run the Bellman-Ford algorithm on the above graph but with a source vertex of 5. Note vertex 1 and 2 are not reachable from 5 and thus should have a distance of infinity. The result should be {1: inf, 2: inf, 3: 2, 4: -3, 5: 0}.

In [95]:
result = BellmanFord({1,2,3,4,5}, {(1, 2, -1), (1, 3, 4), (2, 4, 2), (2, 5, 2), (5, 4, -3), (4, 3, 5), (2, 3, 3)}, 5)
print(result)

{1: inf, 2: inf, 3: 2, 4: -3, 5: 0}


Note, this will return none if there is a negative cycle. For instance, in the above graph if we had an edge from 4 going back to 2 with weight -1 and all of the other edges were -1 this would form a negative cycle. Here is the example:

In [96]:
result = BellmanFord({1,2,3,4,5}, {(1, 2, -1), (1, 3, -1), (2, 4, -1), (2, 5, -1), (5, 4, -1), (4, 3, -1), (4,2,-1), (2, 3, -1)}, 1)
print(result)

None


### Johnson's algorithm

One way to solve the APSP problem is by running the Bellman-Ford algorithm from each vertex. If there are n vertices and m edges the total runtime for this algorithm would be O(n^2m) since Bellman-Ford is O(nm). But since Dijsktra's is O(mlog(n)) which is a quicker runtime than O(nm) it would be great if there was a way to run Dijkstra's algoritgm those n times instead. The issue with just running Dijkstra's is if there are negative edges, this would be an issue and why we can just run Dijkstra's on the graph. On the other hand, Bellman-Ford can work with negative edges. This is where Johnson's algorithm intuition comes from.

In Johnson's algorithm we first get the shortest path length to each vertex using Bellman-Ford's algorithm since we know it works on graphs with negative edge weights. We do this by creating a "dummy" source vertex that will connect to all vertices and run Bellman-Ford from that dummy source vertex. Using these shortest paths, we can modify the weights on the edges and eliminate all the negative weights. After eliminating all these negaitve edges we can run Dijkstra's from each vertex.

But how does adjusting the edge weights work? Won't changing the weights of the edges change the length of the shortest path? 
Suppose we have an edge (u, v), what we would do to the weight of that edge is add the length of the shortest path to u and subtract the length of the shortest path v to the weight of the edges. This is because every vertex entered, we also need to exit it. So even though we are adding the path length of u when leaving vertex u, we will be subtracting the same length when entering the vertex u. Essentially for every x we add to an edge's weight, we also subtract that same x except for the inital and last vertex which we can adjust for after. So in the end we are subtracting every value we add and thus in the end the length of the shortest path does not change. There is a lemma that we will not be going into detail here but this path potentials lemma states for a weighted graph G = (V, E) and any assignment p(v): V -> reals that for w'(u,v) = w(u, v) + p(u) - p(v) and G'=(V,E') with adjusted weights using w'(u,v), we have \[where d_G(u,v) is the distance/path length from u to v in the graph G\]: d_G(u,v) = d_G'(u,v)-p(u)+p(v).

The total runtime is O(nm + mnlog(n)) since we run Bellman-Fords once and Dijkstra's n times.

### Code for Johnson's Algorithm

Now we will code out Johnson's algorithm.

In [108]:
def Johnsons(V, E):
    dummyNode = 0
    newSet = set()
    for edges in E: #Just copies all the edges to a new set to avoid aliasing
        newSet.add(edges)
        
    withDummy = set()
    withDummy.add(dummyNode) # Vertex set with the dummynode
    for vertex in V: # add the dummy node to all the vertices
        newSet.add((dummyNode, vertex, 0))
        withDummy.add(vertex)

    bf = BellmanFord(withDummy, newSet, dummyNode)
    print(bf)
    if bf == None: # This means there was a negative cycle
        return None
    
    newWeights = set()
    for (u,v,w) in E:
        newWeights.add((u, v, w+bf[u]-bf[v]))
    
    print(newWeights)
    sources = {}
    for source in V:
        result = Dijkstra(V, newWeights, source)
        for key in result:
            result[key] = result[key] - bf[source] + bf[key] # Adjust the weights for the first and last nodes
        sources[source] = result
        
    return sources

When running Johnson's algorithm on the above graph we ran Bellman Ford on, we should get the same values as when we ran Bellman-Ford with source vertices 1 and 5.

In [109]:
result = Johnsons({1,2,3,4,5}, {(1, 2, -1), (1, 3, 4), (2, 4, 2), (2, 5, 2), (5, 4, -3), (4, 3, 5), (2, 3, 3)})
for key in result:
    print(f"Source vertex {key} has distances {result[key]}")

{0: 0, 1: 0, 2: -1, 3: 0, 4: -3, 5: 0}
{(4, 3, 2), (1, 2, 0), (2, 3, 2), (1, 3, 4), (2, 5, 1), (2, 4, 4), (5, 4, 0)}
Source vertex 1 has distances {1: 0, 2: -1, 3: 2, 4: -2, 5: 1}
Source vertex 2 has distances {1: inf, 2: 0, 3: 3, 4: -1, 5: 2}
Source vertex 3 has distances {1: inf, 2: inf, 3: 0, 4: inf, 5: inf}
Source vertex 4 has distances {1: inf, 2: inf, 3: 5, 4: 0, 5: inf}
Source vertex 5 has distances {1: inf, 2: inf, 3: 2, 4: -3, 5: 0}


After running Bellman-Ford's with the dummy node, we have that the distances for this example are {0: 0, 1: 0, 2: -1, 3: 0, 4: -3, 5: 0}. To see why create a node connecting to all the other vertices in the graph and run Bellman-Fords on the graph. After this you can adjust all the weights as specified before by adding the distance from Bellman-Ford for u and subtracing the distance for v for an edges (u, v) to the weight w. 

For this example, that is:
(1, 2, -1) -> -1 - (-1) + 0 = 0 -> (1, 2, -1)

(1, 3, 4) -> 4 + 0 - 0 = 4 -> (1, 3, 4)

(2, 4, 2) -> 2 + -1 - (-3) = 4 -> (2, 4, 4)

(2, 5, 2) -> 2 -1 + 0 = 1 -> (2, 5, 1)

(5, 4, -3) -> -3 - (-3) + 0 = 0 -> (5, 4, 0)

(4, 3, 5) -> 5 -3 + 0 = 2 -> (4, 3, 2)

(2, 3, 3) -> 3 -1 + 0 = 2 -> (2, 3, 2)

Now with these new edge weights, we can run Dijkstra's since they are all positive. I will not go through them here but you can follow the process above when running Dijkstra's and adjust the weights by subtracting the distance of v from Bellman Ford and adding the distance of u for each distance for (u, v) to confirm you get the outputted results.

### Further resources
To find addtional information on this topic you can go to the following resources:

https://en.wikipedia.org/wiki/Johnson%27s_algorithm

https://brilliant.org/wiki/johnsons-algorithm/

https://www.scaler.com/topics/data-structures/johnsons-algorithm/

https://www.geeksforgeeks.org/johnsons-algorithm/